# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**: Transformation and actions

**Fecha**: 21/02/2024

**Nombre del Estudiante**:
    - Luis Acosta
    - Samantha Quintero
    - Benjamin Vergara

**Profesor**: Pablo Camarillo Ramirez

In [32]:
import findspark
findspark.init()

In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://be6296989c4d:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [34]:
import importlib
import gatubelxs.spark_utils

importlib.reload(gatubelxs.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = gatubelxs.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

Transformations

In [35]:
from pyspark.sql.functions import desc, sum

In [36]:
#Filter and Count Popular Tracks
popularity_df = tiktok_df.select("popularity")
psongs_df=popularity_df.filter(tiktok_df["popularity"] > 80)

In [37]:
#Calculate Average Duration of Songs by Genre,
genre_df = tiktok_df.select("genre","duration_mins")
grouped_df = genre_df.groupBy("genre").avg("duration_mins")

In [38]:
# Top 5 most energetic songs
energy_def=tiktok_df.select("energy")
sorted_df = energy_def.sort(desc("energy"))

In [39]:
# Total duration of songs in playlists
playlist_id_df = tiktok_df.select("playlist_id","duration_mins")
playlist_df = playlist_id_df.groupBy("playlist_id").sum("duration_mins")

Actions

In [40]:
row_count=psongs_df.count()
row_count

1023

In [41]:
grouped_df.show()

+------------------+------------------+
|             genre|avg(duration_mins)|
+------------------+------------------+
|TIKTOK PHILIPPINES|3.2801328435737513|
|      TIKTOK DANCE| 3.015020713916861|
|           _TIKTOK| 3.251196442168827|
|        TIKTOK OPM| 4.257192861885788|
+------------------+------------------+



In [42]:
sorted_df.show(n=5)
playlist_df.show()

+------------------+
|            energy|
+------------------+
|0.9990000000000001|
|             0.998|
|             0.995|
|             0.995|
|             0.995|
+------------------+
only showing top 5 rows

+--------------------+------------------+
|         playlist_id|sum(duration_mins)|
+--------------------+------------------+
|5IZc3KIVFhjzJ0L2k...| 7.474666666666667|
|08ia51KbTcfs4QVT5...|            4.1485|
|7xVLFuuYdAvcTfcP3...| 9.456433333333333|
|2RBILNmyq8p4fqVWO...| 2.162933333333333|
|6GdDjthxbTGBV9rl2...|3.3209166666666667|
|7krYEnB1OI1RbnJBa...|2.0957666666666666|
|1FgPyHX7HruKDL4Tx...|            2.4448|
|62RtxFf9epYNWOUHJ...|2.6694333333333335|
|5ow0sNF1zSqp71Ix5...|2.7334833333333335|
|0LlJbV4lyzJYE14YC...|10.709133333333334|
|6NFKf8vBApSvtzkap...|3.7074333333333334|
|5P8lyudWE7HQxb4lu...| 4.250666666666667|
|2BgEsaKNfHUdlh97K...| 3.116433333333333|
|7F9vK8hNFMml4GtHs...| 3.173783333333333|
|4vVTI94F9uJ8lHNDW...|3.3657666666666666|
|2uULRpRtKhCdojXwo...|        

In [43]:
sc.stop()